In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report 

In [13]:
set(sns.get_dataset_names())

{'anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic'}

## Метрики классификации
Посмотрим работу с метриками классификации.  
Для этого поугадываем вид пингвина.

In [9]:
df = sns.load_dataset('penguins').dropna()
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


X и y выберем просто 

In [23]:
# не буду включать остров, т.к. это сильная подсказка
df.groupby(['island', 'species']).count()

bill_length_mm  bill_depth_mm  flipper_length_mm  \
island    species                                                       
Biscoe    Adelie                 44             44                 44   
          Gentoo                119            119                119   
Dream     Adelie                 55             55                 55   
          Chinstrap              68             68                 68   
Torgersen Adelie                 47             47                 47   

                     body_mass_g  sex  
island    species                      
Biscoe    Adelie              44   44  
          Gentoo             119  119  
Dream     Adelie              55   55  
          Chinstrap           68   68  
Torgersen Adelie              47   47

In [44]:
X = df.iloc[:, 2:-1]
y = df['species']

Поделим данные на трейн и тест

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [47]:
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier

In [101]:
# сделаем дерево глубиной 5
classifier = DecisionTreeClassifier(max_depth=2)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

Оценим качество классифкатора

In [102]:
y_pred = classifier.predict(X_test)

In [103]:
metrics.accuracy_score(y_test, y_pred)

0.9642857142857143

In [104]:
metrics.precision_score(y_test, y_pred, average='weighted')

0.9685714285714285

In [105]:
metrics.recall_score(y_test, y_pred, average='weighted')

0.9642857142857143

In [106]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.9645751917788391

In [107]:
metrics.roc_auc_score(y_test, classifier.predict_proba(X_test), average='weighted', multi_class='ovr')

0.9893640251799645

## Cross validate
Как использовать обученную модель?

In [3]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

In [28]:
diabetes = datasets.load_diabetes()
X = diabetes.data[:150]
y = diabetes.target[:150]
lasso = linear_model.Lasso()

In [17]:
cv_results = cross_validate(lasso, X, y, cv=3)
cv_results

{'fit_time': array([0.00197244, 0.0010283 , 0.00099921]),
 'score_time': array([0.        , 0.        , 0.00100398]),
 'test_score': array([0.33150734, 0.08022311, 0.03531764])}

In [18]:
lasso.predict(X)

NotFittedError: This Lasso instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Модели как будто нет: "This Lasso instance is not fitted yet". Попробуем предварительно обучить.

In [19]:
lasso.fit(X[:10], y[:10])

Lasso()

In [25]:
cv_results = cross_validate(lasso, X, y, cv=3)
cv_results

{'fit_time': array([0.00099587, 0.00099945, 0.00099659]),
 'score_time': array([0.        , 0.00099683, 0.00100851]),
 'test_score': array([0.33150734, 0.08022311, 0.03531764])}

Предварительное обучение никак не влияет на результат (тест_скор).  
cross_validate сам обучает "сырую" модель и выдает скор, не давая возможности выбрать конкретную модель?
Кажется, нашел:

In [29]:
cv_results = cross_validate(lasso, X, y, cv=3, return_estimator=True)

In [ ]:
# так не работало:
lasso.predict(X)

In [33]:
# а вот так сработает:
cv_results['estimator'][0].predict(X)[0], cv_results['estimator'][1].predict(X)[0], cv_results['estimator'][2].predict(X)[0]

(174.26933995722175, 183.96518603504884, 149.620519140466)

In [16]:
sns.load_dataset('diamonds')

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


## Модели классификации
на датасете с пингвинами

In [30]:
penguins_raw = sns.load_dataset('penguins').dropna()
penguins_raw.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


In [31]:
penguins_raw.species.unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [32]:
# не буду включать остров, т.к. это сильная подсказка
penguins_raw.groupby(['island', 'species']).count()

bill_length_mm  bill_depth_mm  flipper_length_mm  \
island    species                                                       
Biscoe    Adelie                 44             44                 44   
          Gentoo                119            119                119   
Dream     Adelie                 55             55                 55   
          Chinstrap              68             68                 68   
Torgersen Adelie                 47             47                 47   

                     body_mass_g  sex  
island    species                      
Biscoe    Adelie              44   44  
          Gentoo             119  119  
Dream     Adelie              55   55  
          Chinstrap           68   68  
Torgersen Adelie              47   47

In [36]:
penguins = (
    pd.concat([penguins_raw, pd.get_dummies(df.sex)], axis=1)
    .drop(columns=['island', 'sex', 'Male'])
)
penguins.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Female
0,Adelie,39.1,18.7,181.0,3750.0,0
1,Adelie,39.5,17.4,186.0,3800.0,1
2,Adelie,40.3,18.0,195.0,3250.0,1
4,Adelie,36.7,19.3,193.0,3450.0,1
5,Adelie,39.3,20.6,190.0,3650.0,0


In [43]:
X = penguins.iloc[:, 1:]
y = penguins['species']

In [48]:
from sklearn.linear_model import LogisticRegression

In [66]:
clf = LogisticRegression(max_iter=2000)

In [69]:
result_LogisticRegression = cross_validate(clf, X, y, scoring='f1_macro')['test_score']
print(*result_LogisticRegression)

0.9632478632478633 1.0 0.964696223316913 1.0 0.9810169491525423
